# Import required Libraries

In [45]:
import numpy as np
import csv
import time 
# 난수 발생 패턴 고정
np.random.seed(1234)  
def randomize(): np.random(time.time())
    

# Set Hyperparameters

In [46]:
RND_MEAN = 0
RND_STD = 0.0030

learning_rate = 0.001

# Main Function

In [47]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    # 아발로니 데이터셋 로드
    load_abalone_dataset() 
    # 모델 파라미터 초기화
    init_model() 
    # 학습 수행 
    train_and_test(epoch_count, mb_size, report) 

# Data Load Function

In [48]:
def load_abalone_dataset():
    with open('./data/abalone.csv') as csvfile:
        # csv는 엑셀처럼 생김
        csvreader = csv.reader(csvfile) 
        # 첫 행은 읽지 않고 넘김 
        next(csvreader, None) 
        rows = []
        # 행 단위로 데이터를 읽어옴
        for row in csvreader:  
            rows.append(row)
    # global 변수 선언 
    global data, input_cnt, output_cnt 
    # input의 크기와 output의 크기 지정(성별을 3개의 원핫벡터로 표현하므로 인풋 특성이 총 10개가 됨)
    input_cnt, output_cnt = 10, 1 
    data = np.zeros([len(rows), input_cnt+output_cnt])
    
    for n, row in enumerate(rows):
        # 성별을 원핫벡터로 표현 
        if row[0] == 'I': data[n, 0] = 1 
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        # 매 행의 0, 1, 2인덱스는 성별을 나타냄, 3부터는 원래 있던 데이터 복붙 
        data[n, 3:] = row[1:]  
        

# Reset parameters Function

In [49]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    # 가중치를 정규분포 형태에서 뽑아옴 여기서 shape는 (10, 1) 
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])  
    bias = np.zeros([output_cnt])

# Train Function

In [50]:
def train_and_test(epoch_count, mb_size, report):
    # 1 epoch당 진행할 학습 횟수(mb_size에 따라 결정됨)
    step_count = arrange_data(mb_size)
    # 테스트 데이터를 받아옴 
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            # 학습 데이터 받아옴 
            train_x, train_y = get_train_data(mb_size, n)
            # 학습 진행, 리턴 값으로 손실값과 정확도를 받음 
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'.format(epoch+1, np.mean(loss), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\n final test: final accuracy = {:5.3f}'.format(final_acc))

# Get data Function

In [51]:
# 테스트할 데이터와 학습할 데이터를 나누는 함수 
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx 
    # 들어온 데이터의 개수만큼 인덱스 생성 
    shuffle_map = np.arange(data.shape[0])
    # 들어온 데이터를 섞어줌 
    np.random.shuffle(shuffle_map)
    # 20%의 데이터는 테스트를 위해 남겨둠 
    step_count = int(data.shape[0] * 0.8) // mb_size
    # 테스트로 쓸 데이터의 첫번째 인덱스 지정 
    test_begin_idx = step_count * mb_size
    return step_count

# 테스트 데이터를 받아옴 
def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    # 테스트를 시작할 데이터부터 가져옴
    test_data = data[shuffle_map[test_begin_idx:]]   
    # 테스트 데이터, 정답 형식으로 리턴 
    return test_data[:, :-output_cnt], test_data[:, -output_cnt]

# 학습할 데이터를 받아옴 
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    # 학습을 처음 진행하는 경우 학습 데이터끼리 순서를 한 번 더 섞어줌(없어도 돌아가기는 하는 코드)
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    # mb_size 만큼의 데이터만 가져옴 
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]] 
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]


# Run Function

In [52]:
# x는 들어온 데이터, y는 정답 
def run_train(x, y):
    # output은 예측한 정답, aux_nn은 역전파 계산을 위해 남겨둔 x 
    output, aux_nn = forward_neural(x)
    # 예측값과 정답의 평균제곱오차와 오차로 리턴 
    loss, aux_pp = forward_post(output, y)
    accuracy = eval_accuracy(output, y)
    
    # dL/dL = 1
    G_loss = 1.0 
    G_output = backprop_post(G_loss, aux_pp)
    backprop_neural(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neural(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

# Forward & Backward Propagation Function

backprop_post의 이해를 돕기 위한 수식이다. 

$$ L = \frac{\sum_{i=1}^M \sum_{j=1}^N square_{ij}}{MN} $$  

$$ \frac{\partial{L}}{\partial{output_{ij}}} = \frac{\partial{L}}{\partial{L}} \times \frac{\partial{L}}{\partial{square_{ij}}} \times \frac{\partial{square_{ij}}}{\partial{diff_{ij}}} \times \frac{\partial{diff_{ij}}}{\partial{output_{ij}}}$$   

$$ square_{ij} = diff_{ij}^2  -> \frac{\partial{square_{ij}}}{\partial{diff_{ij}}} = 2diff_{ij}$$     

$$ \frac{\partial{L}}{\partial{square_{ij}}} = \frac{1}{MN} $$   

$$ \frac{\partial{diff_{ij}}}{\partial{output_{ij}}} = 1 $$ 


다음은 backprop_neural을 이해하기 위한 수식이다. 
$$ \frac{\partial{L}}{\partial{W_{kj}}} = \frac{\partial{L}}{\partial{output_{ij}}} \times \frac{\partial{output_{ij}}}{\partial{W_{kj}}} $$   
$$ \frac{\partial{L}}{\partial{W}} = X^T G (\frac{\partial{L}}{\partial{output}} = G) $$  
$$ \frac{\partial{L}}{\partial{B_j}} = \sum_{k=1}^m G_{kj} $$

In [53]:
def forward_neural(x):
    global weight, bias
    # mb_size가 10이므로 x가 (10, 10)이고 weight가 (10, 1) 따라서 output은 (10, 1)이 됨. 
    output = np.matmul(x, weight) + bias
    return output, x

# 계산한 미분값을 이용해 w와 b를 업데이트 해주는 함수 
def backprop_neural(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)
    
    weight -= learning_rate * G_w
    bias -= learning_rate * G_b
    
# 평균제곱 오차와 역전파 계산을 위해 그냥 오차 값을 전달하는 함수 
def forward_post(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

# 역전파 계산을 위해 dL/dY 값을 구해주는 함수 
def backprop_post(G_loss, diff):
    shape = diff.shape # (10, 1)
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1
    
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

# Accuracy Function

In [54]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output-y)/y))
    return 1 - mdiff

# Run

In [55]:
abalone_exec()
print(weight)
print(bias)

Epoch 1: loss=10.241, accuracy=0.557/0.692
Epoch 2: loss=15.216, accuracy=0.820/0.674
Epoch 3: loss=4.751, accuracy=0.812/0.672
Epoch 4: loss=8.049, accuracy=0.808/0.676
Epoch 5: loss=2.982, accuracy=0.810/0.677
Epoch 6: loss=20.395, accuracy=0.808/0.679
Epoch 7: loss=9.585, accuracy=0.808/0.681
Epoch 8: loss=7.799, accuracy=0.808/0.684
Epoch 9: loss=3.493, accuracy=0.810/0.683
Epoch 10: loss=3.433, accuracy=0.809/0.684

 final test: final accuracy = 0.684
[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]
